# 💿 Подключение к Google Drive

Перед началом работы необходимо подключить Google Drive. Установите значение переменной PROJECT:

## Метод 1, через Secret:

1. В левой панели перейдите на вкладку "Ключ" (🔑)
2. Добавьте следующие секрет:
    - `PROJECT`: `/content/drive/<путь к папке на Google Drive>`

## Метод 2, напрямую в ячейке с кодом:

`PROJECT = '/content/drive/<путь к папке на Google Drive>'`

После этого запустите ячейку с кодом.

In [1]:
# Mount Google Drive and setup paths
PROJECT = ''

if PROJECT == '':
    try:
      from google.colab import userdata
      PROJECT = userdata.get('PROJECT')
    except:
      print('Секрет PROJECT не установлен')
      raise

if str.isspace(PROJECT) or PROJECT == '':
  raise Exception("Не удалось установить значение переменной PROJECT")

from google.colab import drive
drive.mount('/content/drive')

# Add src directory to Python path
import sys
if f'{PROJECT}/src' not in sys.path:
    sys.path.insert(0, f'{PROJECT}/src')

# Change to project directory
%cd {PROJECT}

# Verify path is added
import os
print(f"✅ Added to sys.path: {PROJECT}/src")
print(f"📁 Current directory: {os.getcwd()}")
print(f"🔍 Available modules in src: {os.listdir('src') if os.path.exists('src') else 'src not found'}")

!pip -q install -U pip
!pip -q install -r requirements.txt


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/Ноутбук/colab-training
✅ Added to sys.path: /content/drive/Othercomputers/Ноутбук/colab-training/src
📁 Current directory: /content/drive/Othercomputers/Ноутбук/colab-training
🔍 Available modules in src: ['data', 'models', 'training', 'inference', 'utils', 'analysis', '__pycache__', '__init__.py', 'config.py']


# 🔑 Настройка переменных окружения

Перед запуском блокнота необходимо настроить переменные окружения:

1. Перейдите к иконке ключа (🔑) в левой боковой панели
2. Добавьте следующие секреты:
   - `SUPABASE_URL`: URL вашего проекта Supabase
   - `SUPABASE_PUBLISHABLE_KEY`: Ваш анонимный ключ Supabase
   - `EMAIL`: Ваш email для входа
   - `PASSWORD`: Ваш пароль для входа

# ⚙️ Конфигурация

## Установка путей

- LMDB_PATH: путь к файлу .lmdb - итоговый датасет
- MAP_PATH: путь к файлу .json - словарь для соответствия подписи в lmdb и её записи в базе данных

In [2]:
# Установка путей
LMDB_PATH = '/content/drive/MyDrive/datasets/v1.lmdb'
MAP_PATH = '/content/drive/MyDrive/datasets/v1.json'

# Фильтрация подписей при создании датасета
INPUT_TYPE = 'any'  # Фильтр по типу ввода: 'any', 'mouse', 'touch', или 'pen'


In [3]:
# 🚀 Обучение
OUTPUT_DIR = "/content/drive/MyDrive/runs"

# Рекомендации диагностики LMDB

from config import DatasetConfig, ModelConfig, TrainingConfig
from training import TrainingRunner

print("⚙️ Configuring Hybrid model training...")

FEATURE_PIPELINE = [
    "vx",
    "vy",
    "ax",
    "ay",
    "prate",
    "path_tangent_angle",
    "abs_delta_pressure",
]

print(f"✅ Using {len(FEATURE_PIPELINE)} derived features: {FEATURE_PIPELINE}")

# Dataset configuration
ds_cfg = DatasetConfig(
    lmdb_path = LMDB_PATH,
    feature_pipeline = FEATURE_PIPELINE,
)

# Model configuration
model_cfg = ModelConfig(

)

# Training configuration
train_cfg = TrainingConfig(
  output_dir = OUTPUT_DIR,
)

print("🚀 Starting Hybrid Model Training\n")

try:
    runner = TrainingRunner(
        dataset_cfg=ds_cfg, model_cfg=model_cfg, train_cfg=train_cfg
    )
    runner.run()

    print("\n" + "=" * 80)
    print("✅ Training completed successfully!")
    print("=" * 80)
    print(f"\n📁 Artifacts saved to: {OUTPUT_DIR}/[run_name or TIMESTAMP]/")
    print(f"  • checkpoints/ - Model checkpoints")
    print(f"  • logs/ - Training metrics")
    print(f"  • exports/ - Plots and exports")

except KeyboardInterrupt:
    print("\n⚠️ Training interrupted by user")

except Exception as e:
    print("\n" + "=" * 80)
    print("❌ Training failed with error:")
    print("=" * 80)
    print(f"\n{type(e).__name__}: {e}\n")
    import traceback

    traceback.print_exc()
    exit(1)

IndentationError: unexpected indent (runner.py, line 560)

In [ ]:

# Проверка готовности

# Проверка работы импортов
try:
    import analysis
    import data
    import models
    import training
    import utils
    import config
    print("✅ Все модули успешно импортированы!")
except ImportError as e:
    print(f"❌ Ошибка импорта: {e}")
    print("Доступные пути:", sys.path)

# Проверка переменных окружения
from google.colab import userdata
print(f"\n🔑 SUPABASE_URL: {'✅ Установлена' if userdata.get('SUPABASE_URL') else '❌ Отсутствует'}")
print(f"🔑 SUPABASE_PUBLISHABLE_KEY: {'✅ Установлена' if userdata.get('SUPABASE_PUBLISHABLE_KEY') else '❌ Отсутствует'}")
print(f"🔑 EMAIL: {'✅ Установлена' if userdata.get('EMAIL') else '❌ Отсутствует'}")
print(f"🔑 PASSWORD: {'✅ Установлена' if userdata.get('PASSWORD') else '❌ Отсутствует'}")


# Вывод конифигураций
from os import path
print(f"\nLMDB_PATH: {LMDB_PATH} ({"существует" if path.exists(LMDB_PATH) else "не существует"})")
print(f"MAP_PATH: {MAP_PATH} ({"существует" if path.exists(MAP_PATH) else "не существует"})")
print(f"INPUT_TYPE: {INPUT_TYPE}")


In [ ]:
# Создание датасета

from data import build_lmdb_from_supabase
from google.colab import userdata
build_lmdb_from_supabase(
    supabase_url=userdata.get('SUPABASE_URL'),
    anon_key=userdata.get('SUPABASE_PUBLISHABLE_KEY'),
    email=userdata.get('EMAIL'),
    password=userdata.get('PASSWORD'),
    output_lmdb_path=LMDB_PATH,
    output_map_json_path=MAP_PATH,
    input_type=INPUT_TYPE,
)
print('LMDB записан в', LMDB_PATH)
print('MAP записан в', MAP_PATH)


In [ ]:
# Отчет по Supabase
from analysis import analyze_data, DataSource, print_supabase_report
print("Анализ...")
supabase_results = analyze_data(DataSource.SUPABASE,
    supabase_url=userdata.get('SUPABASE_URL'),
    anon_key=userdata.get('SUPABASE_PUBLISHABLE_KEY'),
    email=userdata.get('EMAIL'),
    password=userdata.get('PASSWORD')
)
print("=== ОТЧЕТ ПО SUPABASE ===")
print_supabase_report(supabase_results)


In [ ]:
# Отчет по LMDB
from analysis import analyze_data, DataSource, print_lmdb_report
print("Анализ LMDB...")
lmdb_results = analyze_data(DataSource.LMDB, lmdb_path=LMDB_PATH)
print("=== ОТЧЕТ ПО LMDB ===")
print_lmdb_report(lmdb_results)


In [ ]:
# 📊 Визуализация распределения t для LMDB
from analysis import plot_t_distribution_by_label

fig, text = plot_t_distribution_by_label(LMDB_PATH, bin_width=500, text_step=100, overlay=True)
fig.show()
print(text)


In [ ]:
# 🔍 Поиск неподрезанных подписей с мусорными точками
from analysis import find_untrimmed_signatures

problematic = find_untrimmed_signatures(LMDB_PATH, max_skip_check=10)
print(f"Найдено {len(problematic)} подозрительных подписей")
# Выведем первые 10 результатов
for rec in problematic[:10]:
    print(rec)
